## 8.3 Newton's Method

**Implementation 8.14: Newton's Method**

For a scalar function, we can implement Newton's method as follows. To study the convergence behaviour, we print some additional information in every step. 

In [ ]:
def newton(f, df, x, n=10, tol=1e-10):
    print('k x_k         f(x_k)')
    print(f'0 {x: .8f} {f(x): .4e}')
    for i in range(n):
        x -=  f(x) / df(x)
        print(f'{i + 1} {x: .8f} {f(x): .4e}')
        if abs(f(x)) < tol:
            print(f"Newton's method converged after {i + 1} iterations")
            return x
    print(f"Newton's method did not converge after {n} iterations")
    print(f'x = {x}, abs(f(x)) = {abs(f(x))} > {tol}')
    return x

#### Example 8.19

We test the method by searching for the roots of the function
$$f(x) = x(1 + \exp(x)) + 10\sin(3 + \log(x^2 + 1)).$$
To this end, we implement the functions and its first derivative.

In [ ]:
from math import exp, log, cos, sin

def f(x):
    return x * (1 + exp(x)) + 10 * sin(3 + log(x**2 + 1))

def df(x):
    return (x**2 + 20 * x * cos(log(x**2 + 1) + 3) + 1)/(x**2 + 1) + exp(x) * (x + 1)

Using the initial value $x_0=0$, we get

In [ ]:
newton(f, df, 0, n=10, tol=1e-12)

We found the root with a residual of machine precision in 6 steps. This is less than a fifth of the number of steps needed by the bisection method.

If the function has multiple roots, it can happen that the method converges to a different root if a different initial guess is used:

In [ ]:
newton(f, df, -10, n=10, tol=1e-12)

When considering vector valued problems, dividing by the derivative turns into solving a linear system (i.e., 'inverting' the Jacobian), which is generally an expensive operation. Therefore, it is interesting to consider Newton-like methods where the Jacobian is approximated. Essentially any such method where some approximation of the Jacobian (derivate) is used is known as a quasi-Newton method.

**Implementation 8.28: Simplified Newton’s Method**

In [ ]:
def newton_simple(f, df, x, c, n=10, tol=1e-10):
    df_inv =  1 / df(c)
    
    print('k x_k          f(x_k)')
    print(f'0 {x: .8f} {f(x): .4e}')
    for i in range(n):
        x -=  df_inv * f(x) 
        print(f'{i + 1:02d} {x: .8f} {f(x): .4e}')
        if abs(f(x)) < tol:
            print(f'The simplied Newton method converged after {i + 1} iterations')
            return x
    print(f'The simplied Newton method did not converge after {n} iterations')
    print(f'x = {x}, abs(f(x)) = {abs(f(x))} > {tol}')
    return x

#### Example 8.30 (Simplified Newton Method)

Applied to the previous exampe with $c=x_0$ and the larger tolerance $\epsilon=10^{-7}$, we get

In [ ]:
newton_simple(f, df, -10, -10, n=50, tol=1e-7)

The method only converges slowly. With a better choice of $c$, we can get faster convergence. However, if $c$ is chosen badly, the method may not converge at all. Try some different values of $c$ for yourself!

In [ ]:
newton_simple(f, df, -10, -9.344, n=50, tol=1e-7)

Rather than choosing a fixed value for the direction of decent, we can also approximate the derivative by a finite difference.

**Implementation 8.32: Approximated Newton Method** 

In [ ]:
def newton_approx(f, x, eps, n=10, tol=1e-10):
    print('k x_k         f(x_k)')
    print(f'0 {x: .8f} {f(x): .4e}')
    for i in range(n):
        y = f(x)
        z = f(x + eps)
        x -=  eps * y / (z - y)
        print(f'{i + 1} {x: .8f} {f(x): .4e}')
        if abs(f(x)) < tol:
            print(f'The approximated Newton method converged after {i + 1} iterations')
            return x
    print(f'The approximated Newton method did not converge after {n} iterations')
    print(f'x = {x}, abs(f(x)) = {abs(f(x))} > {tol}')
    return x

#### Example 8.33 (Approximated Newton Method)

Here, the rate of convergence depends on the choice of $\epsilon$ in the computation of the finite difference.

In [ ]:
for eps in [1e-1, 1e-2, 1e-4, 1e-8, 1e-12, 2e-14]:
    print('\n------------------------')
    print(f'eps = {eps}\n')
    newton_approx(f, -10, eps, n=10, tol=1e-10)

Finally, we consider some less well-known algorithms to compute the root of a function

**Implementation of Algorithm 8.34: Secant Method**

In [ ]:
def secant(f, x0, x1, n=10, tol=1e-12):
    f0 = f(x0)
    for i in range(n):
        f1 = f(x1)
        if abs(f1) < tol:
            print(f'The Secant method converged after {i + 1} iterations')
            return x1
        
        x2 = x1 - f1 * (x1 - x0) / (f1 - f0)
        x0, f0 = x1, f1
        x1 = x2
    print(f'The Secant method did not converge after {n} iterations')
    print(f'x = {x1}, abs(f(x)) = {abs(f1)} > {tol}')
    return x1

Applied to our previous example with $x_0=-10$ and $x_1=-9$, we get

In [ ]:
secant(f, -10, -9, n=10, tol=1e-12)

**Implementation of Algorithm 8.34: Regula falsi**

In [ ]:
def regula_falsi(f, x0, x1, n=10, tol=1e-12):
    f0 = f(x0)
    f1 = f(x1)
    for i in range(n):
        
        if abs(f1) < tol:
            print(f'The Regula falsi method converged after {i + 1} iterations')
            return x1
        
        x2 = x1 - f1 * (x1 - x0) / (f1 - f0)
        f2 = f(x2)
        if f2 * f1 < 0:
            x0, f0 = x1, f1
            x1, f1 = x2, f2
        else:
            x1, f1 = x2, f2
    print(f'The Regula falsi method converged after {n} iterations')
    print(f'x = {x1}, abs(f(x)) = {abs(f1)} > {tol}')
    return x1

Using the same parameters as used for the Secant method, we get

In [ ]:
regula_falsi(f, -10, -9, n=50, tol=1e-12)

We see that we have indeed lost the fast convergence of the secant method.